<h1>Convolutional Neural Networks anhand von MNIST</h1>

<p>Das neuronale Netzwerk der letzten &Uuml;bung brauchte nur einige hundert Megabyte Arbeitsspeicher. Die 60000 MNIST Bilder mit ihren 784 Pixeln, die Gewichtsmatrizen und alle Zwischenergebnisse, die im Netzwerk beim Vorw&auml;rtspass berechnet wurden, sind keine 350MB gro&szlig;:<br />
(60000&lowast; 784 + 784&lowast; 300 + 60000 &lowast; 300 + 300 &lowast; 10 + 60000 &lowast; 10) &lowast; 4 = 335MB <br />(MNISTData + Weights1 + Intermediate + Weight2 + Predictions) &lowast; Float32</p>

<p>Dieser Verbrauch steigt rasant an, wenn Konvolutionsfilter ins Spiel kommen. Werden die Eingangsdaten mit 10 Filtern beliebiger Gr&ouml;&szlig;e (z.b. 3x3) gefalten, sind die ausgehenden Daten 10 mal so gro&szlig;:<br />
(60000&lowast;784+10&lowast;3&lowast;3+60000&lowast;10&lowast;784)&lowast;4=2GB</p>

<p>Um das zu verhindern, sollten in Zukunft nicht mehr alle Daten auf einmal im Netzwerk verarbeitet werden. Stattdessen werden Mini-Batches ben&ouml;tigt.</p>

<p>Das komplette Notebook steht wieder zum <a href="06_Tensorflow_ConvNet_MNIST_Vorlage.ipynb">download</a> bereit.</p>

<hr>

<h2>Neural Networks mit Mini-Batches</h2>

<p>Im folgenden ist ein zweischichtiges neuronales Netzwerk implementiert, welches alle MNIST Ziffern auf einmal verarbeitet. Bauen Sie den Code so um, dass er stattdessen mit Mini-Batches funktioniert. Dabei&nbsp;k&ouml;nnen Sie Numpy verwenden und die Daten als Mini-Batch in den Computation-Graph von Tensorflow geben&nbsp;oder Sie benutzen Tensorflows Batch-Methoden, um die Batches innerhalb eines Graphens zu erzeugen. Die K&ouml;nigsdiziplin sind Tensorflow Esitmators, die die Arbeit des Batchings &uuml;bernehmen, aber viele andere Anforderungen an das Netzwerk haben.&nbsp;Wichtig ist in allen&nbsp;F&auml;llen, dass auch die Testdaten gebatched werden.</p>

<ul>
	<li><strong>Numpy</strong>: Es ist m&ouml;glich die Daten einmalig in kleine Batches zu unterteilen und diese dann in zuf&auml;lliger Reihnfolge in den Computation-Graph zu geben. Besser jedoch ist die Variante, bei der erst im letzten Moment ein Batch aus dem gesamten Datensatz extrahiert wird. Der Extraktionsbereich sollte dabei zuf&auml;llig gew&auml;hlt sein.&nbsp;&nbsp;</li>
	<li><strong>Tensorflow <a href="https://www.tensorflow.org/guide/datasets" target="_blank">Dataset</a></strong>: Sind die&nbsp;Daten klein genug, dass Sie&nbsp;in den Arbeitsspeicher, aber nicht mit einen Durchlauf durch das Netzwerk passen,  k&ouml;nnen sie zun&auml;chst&nbsp;komplett in den Graphen geladen werden und von dort in <a href="https://www.tensorflow.org/guide/datasets#batching_dataset_elements" target="_blank">kleine Batches</a> zerlegt werden. Mithilfe von <a href="https://www.tensorflow.org/guide/datasets#creating_an_iterator"  target="_blank">Iteratoren</a> können diese Batches dann einzeln in das Netzwerk geschickt werden.</li> 
<li><strong>Tensorflow Estimator</strong>: Innerhalb der High-Level API von Tensorflow gibt es die Möglichkeit, Estimators zu verwenden. Diese übernehmen sämtliche Batching-Arbeiten, verlangen aber bestimmte Eigenschaften vom Computation-Graphen. So m&uuml;ssen Trainings- und Evaluierungsmethoden in einen sogenannten <a href="https://www.tensorflow.org/api_docs/python/tf/estimator/EstimatorSpec">EstimatorSpec</a> beschrieben werden, um sp&auml;ter mit einen <a href="http://www.tensorflow.org/api_docs/python/tf/estimator">Estimator Model</a> arbeiten zu k&ouml;nnen.</li>
</ul>

In [ ]:
!pip install tensorflow-gpu==1.15.0
!pip install deep-teaching-commons

In [ ]:
import tensorflow as tf
import numpy as np
import time
from matplotlib import pyplot as plt
from tqdm import tqdm
from shutil import copyfileobj
from sklearn.datasets.base import get_data_home
from deep_teaching_commons.data.fundamentals import mnist
from sklearn.utils import check_random_state
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_train, y_train, X_test, y_test = mnist.Mnist().get_all_data(normalized=True, flatten=False)

X_train = X_train.reshape((-1, 28, 28, 1))
X_test = X_test.reshape((-1, 28, 28, 1))

# only shuffle train dataset
random_state = check_random_state(0)
permutation = random_state.permutation(X_train.shape[0])
X_train = X_train[permutation]
y_train = y_train[permutation]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

enc = OneHotEncoder()

y_train = enc.fit_transform(np.expand_dims(y_train, axis=1)).toarray()
y_test = enc.fit_transform(np.expand_dims(y_test, axis=1)).toarray()
print(y_train.shape, y_test.shape)

In [ ]:
def minibatcher(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
    yield inputs[excerpt], targets[excerpt]

<hr>

<h2>Convolutional Neural Network mit MNIST Ziffern</h2>

<p>Nachdem das neuronale Netzwerk mit Mini-Batches arbeitet, k&ouml;nnen die Fully-Connected (Dense) Layer mit Konvolutionsschichten ersetzt werden. Sinnvoll sind z.B. zwei Schichten mit 64 5x5 und 96 3x3 Filterkerneln. Um die Dimensionalit&auml;t der Daten langsam zu reduzieren, k&ouml;nnen entweder Schrittweiten bei den Konvolutionsschichten eingestellt werden oder Pooling angewendet werden. Zum Schluss ist es hilfreich, die hochdimensionalen Daten zu flatten, um sie in Dense Layern auf 10 Dimensionen herunterzubrechen. Berechnen Sie wieder den Trainingsfehler und die Testgenauigkeit. Zu erwarten sind Genauigkeiten von bis zu 99%.&nbsp;</p>

<p>Je nachdem welche Tensorflow Version Sie nutzen (mindestens aber Version &gt;= 1.0), sind folgende Methoden hilfreich:</p>

<ul>
	<li><a href="https://www.tensorflow.org/api_docs/python/tf/nn/max_pool" target="_blank">tf.nn.max_pool</a> oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/max_pooling2d" target="_blank">tf.layers.max_pooling2d</a></li>
	<li><a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/conv2d" target="_blank">tf.nn.conv2d</a> oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/conv2d" target="_blank">tf.layers.conv2d</a></li>
	<li><a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/reshape" target="_blank">tf.reshape</a>  oder <a href="https://www.tensorflow.org/versions/master/api_docs/python/tf/layers/flatten" target="_blank">tf.layers.Flatten</a></li>
</ul>

<p><strong>Optional</strong>: Yann LeCun hat vor fast 20 Jahren das MNIST Datenset herausgebracht und die Convolutionsnetzwerke erfunden. Damals gab es nicht die nötige Rechenleistung um in kurzer Zeit die notwendigen Filterkernel mittels Backpropagation und Gradient Descent zu erlernen. Seine Netzwerke sind daher sehr minimalistisch. Implementieren Sie das <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf" target="_blank">LeNet5</a> Netzwerk nach seinen Vorbild. Padden Sie dazu die&nbsp;Eingangsdaten, damit die Bilder 32x32 Pixel haben und verwenden Sie nur 6, 16 und 120 Filterkernel&nbsp;(je 5x5 Pixel gro&szlig;) f&uuml;r die drei Konvolutionsschichten in LeNet5. Natürlich können Sie auch LeCun's <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf" target="_blank">Stochastic gradient descent</a> nutzen um ihr Netzwerk zu trainieren oder gar den <a href="https://arxiv.org/pdf/1412.6980v8.pdf" target="_blank">Adam Optimizer</a>.</p>

<p>&nbsp;</p>

![LeNet5.png](attachment:LeNet5.png)

In [ ]:
# pixel count
num_input = 28

# num of classes
num_classes = 10

# learn rate
learning_rate = 0.001

batch_size = 128

In [ ]:
# computation graph
graph = tf.Graph()
with graph.as_default():
    
    # input data with fix shape to infer shapes of other graph nodes a build time
    x_input = tf.placeholder(dtype=tf.float32, shape=[None, num_input, num_input, 1], name='x')
    y_input = tf.placeholder(tf.int64, shape=[None, num_classes], name='y')
        
    layer1 = tf.layers.conv2d(inputs=x_input, filters=64, kernel_size=(5,5), activation=tf.nn.relu)
    layer2 = tf.layers.conv2d(inputs=layer1, filters=64, kernel_size=(5,5), activation=tf.nn.relu)
    flatten = tf.layers.flatten(layer2)
    dense = tf.layers.dense(inputs=flatten, units=128, activation=tf.nn.relu)
    prediction = tf.layers.dense(inputs=dense, units=num_classes)
    
    # compute trainings error
    cost = tf.losses.softmax_cross_entropy(onehot_labels=y_input, logits=prediction)
    
    # use the Adam optimizer to derive the cost function and update the weights
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # accuracy for multiple batches
    softmax = tf.nn.softmax(prediction)
    acc, update_acc = tf.metrics.accuracy(labels=tf.argmax(y_input, 1), predictions=tf.argmax(softmax, axis=-1))

In [ ]:
# start a new session
with tf.Session(graph=graph) as session:  
    # initialize weights and bias variables
    session.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))     

    # check against test set
    print("Test accuracy ", session.run(update_acc, feed_dict={x_input: X_test, y_input: y_test}))

    # reset accuracy
    session.run(tf.local_variables_initializer())   

    # train for a few iterations
    ts = time.time()
    train_errors = []
    for i in range(100):
        train_batcher = minibatcher(X_train, y_train, batch_size)
        temp = []
        for i, (X_batch, y_batch) in enumerate(train_batcher):
            c, _ = session.run([cost, optimizer], feed_dict={x_input: X_batch, y_input: y_batch})
            temp.append(c)
        train_errors.append(np.mean(temp))
    print("Improved train error from ", train_errors[0], " to ", train_errors[-1], " in ", str(time.time()-ts), "secs")

    test_batcher = minibatcher(X_test, y_test, batch_size)
    for i, (X_batch, y_batch) in enumerate(test_batcher):
         session.run(update_acc, feed_dict={x_input: X_batch, y_input: y_batch})

    # check against test set
    print("Test accuracy ", session.run(acc))

In [ ]:
# plot the train errors
plt.plot(train_errors)
plt.show()

<hr />

<h2>Abgabe</h2>

Bevor sie das Notebook in Moodle hochladen entfernen sie bitte über "Kernel" -> "Restart and Clear Output" sämtlichen von Python erstellten Inhalt und speichern anschließend das Notebook "File" -> "Save and Checkpoint" erneut ab. Sorgen sie bitte außerdem dafür das im Dateinamen ihr Vor- und Nachname steht, ich empfehle folgende Namensgebung: "06_Tensorflow_ConvNet_MNIST_VORNAME_NACHNAME.ipynb"